In [37]:
import os
import glob
from functools import reduce
import operator

In [38]:
for file in glob.glob('stl_files/*.stl'):
  print(file)

stl_files/cube.stl
stl_files/largecube.stl
stl_files/monkey.stl


In [42]:
import itertools as it
import numpy as np
import pandas as pd
from subprocess import check_output,CalledProcessError

"""
this script automates the process of slicing the same stl file with many possible combinations of command line arguments 
that can be passed to slic3r
"""
def flag2placeholder(flag):
  flag_str = str(flag)
  flag_str_clean = flag_str.strip("-").replace("-","_")
  
  return flag_str_clean+"["+flag_str_clean+"]"

#dict of lists of possible arguments to be tested
configurations = {"--nozzle-diameter":[0.5,1.0],                  
                  "--use-firmware-retraction":[True,False],
                  "--use-volumetric-e":[True,False],                   
                  "--vibration-limit":[0,5,10],                   
                  "--filament-diameter":[2,3,4],
                  "--extrusion-multiplier":[0.9,1,1.1],                  
                  "--layer-height":[0.3,0.4,0.5],                  
                  "--infill-every-layers":[1,3,5],                  
                  "--perimeters":[0,1],
                  "--solid-layers":[0,1],
                  "--fill-density":[10,20,50],
                  "--fill-angle":[30,45,60],
                  "--fill-pattern":["Octagram Spiral"],
                  #"--external-fill-pattern":[],                  
                  #"--end-gcode":[],
                  #"--before-layer-gcode":[],
                  #"--layer-gcode":[],
                  #"--toolchange-gcode":[],
                  #"--seam-position":[],
                  #"--external-perimeters-first":[],
                  #"--spiral-vase":[],
                  #"--only-retract-when-crossing-perimeters":[],
                  #"--solid-infill-below-area":[],
                  #"--infill-only-where-needed":[],
                  #"--infill-first":[],
                  #"--extra-perimeters":[],
                  #"--avoid-crossing-perimeters":[],
                  #"--thin-walls":[],
                  #"--overhangs":[],
                  #"--support-material":[],
                  #"--support-material-threshold":[],
                  #"--support-material-pattern":[],
                  #"--support-material-spacing":[],
                  #"--support-material-angle":[],
                  #"--support-material-contact-distance":[],
                  #"--support-material-interface-layers":[],
                  #"--support-material-interface-spacing":[],
                  #"--raft-layers":[],
                  #"--support-material-enforce-layers":[],
                  #"--dont-support-bridges":[],
                  #"--retract-length":[],
                  #"--retract-speed":[],
                  #"--retract-restart-extra":[],
                  #"--retract-before-travel":[],
                  #"--retract-lift":[],
                  #"--retract-layer-change":[],
                  #"--wipe":[],
                  #"--cooling":[],
                  #"--min-fan-speed":[],
                  #"--max-fan-speed":[],
                  #"--bridge-fan-speed":[],
                  #"--fan-below-layer-time":[],
                  #"--slowdown-below-layer-time":[],
                  #"--min-print-speed":[],
                  #"--disable-fan-first-layers":[],
                  #"--fan-always-on":[],
                  #"--skirts":[],
                  #"--skirt-distance":[],
                  #"--skirt-height":[],
                  #"--min-skirt-length":[],
                  #"--brim-width":[],
                  #"--scale":[],
                  #"--rotate":[],
                  #"--duplicate":[],
                  #"--duplicate-grid":[],
                  #"--duplicate-distance":[],
                  #"--xy-size-compensation":[],
                  #"--complete-objects":[],
                  #"--extruder-clearance-radius":[],
                  #"--extruder-clearance-height":[],
                  #"--notes":[],
                  #"--resolution":[],
                  #"--extrusion-width":[],
                  #"--first-layer-extrusion-width":[],
                  #"--perimeter-extrusion-width Set a different extrusion width for perimeters":[],
                  #"--external-perimeter-extrusion-width":[],
                  #"--infill-extrusion-width":[],
                  #"--solid-infill-extrusion-width":[],
                  #"--top-infill-extrusion-width":[],
                  #"--support-material-extrusion-width":[],
                  #"--infill-overlap":[],
                  #"--bridge-flow-ratio Multiplier for extrusion when bridging (> 0, default: 1)":[],
                  #"--extruder-offset":[],
                  #"--perimeter-extruder":[],
                  #"--infill-extruder":[],
                  #"--solid-infill-extruder":[],
                  #"--support-material-extruder":[],
                  #"--support-material-interface-extruder Extruder to use for support material interface (1+, default: 1)":[],
                  #"--ooze-prevention":[],
                  #"--standby-temperature-delta":[],
                  #"--ooze-prevention is enabled (default: -5)":[], 
                  #"--retract-length-toolchange":[],
                  #"--retract-restart-extra-toolchange":[]                  
                 }


combinations = it.product(*(configurations[Name] for Name in configurations))
count=1
total=len(list(combinations))
#loop through every combination of arguments
input_file = os.path.abspath("stl_files/largecube.stl")
for configuration in list(it.product(*configurations.values())):
    #construct a list of command line arguments   
    output_file_format="[input_filename_base]"
    print("{} out of {}".format(count,total))    
    cmd=["slic3r"]
    for key,value in zip(configurations.keys(),configuration):
      print("adding {} with value of {} to cmd".format(key,value))
      if value:        
        cmd.append(str(key))
        if not isinstance(value,bool):
          cmd.append(str(value))     
        output_file_format+="_"+flag2placeholder(key)
    cmd.append("--output-filename-format")
    cmd.append(output_file_format+".gcode")
    cmd.append(input_file)    
    
    cmd_str=''
    for arg in cmd:
      
      cmd_str += ' '+str(arg)         
    print(cmd_str)
    
    #run the command and error if the exit status is non-zero
    try:
      count+=1
      check_output(cmd)      
    except CalledProcessError as e:
      print("unable to slice with error: {}".format(e))
      continue
    #print('done')
    

1 out of 24
adding --nozzle-diameter with value of 0.5 to cmd
adding --use-firmware-retraction with value of True to cmd
adding --use-volumetric-e with value of True to cmd
adding --vibration-limit with value of 0 to cmd
 slic3r --nozzle-diameter 0.5 --use-firmware-retraction --use-volumetric-e --output-filename-format [input_filename_base]_nozzle_diameter[nozzle_diameter]_use_firmware_retraction[use_firmware_retraction]_use_volumetric_e[use_volumetric_e].gcode /home/thom/PycharmProjects/optimization_of_3d_printable_objects/stl_files/largecube.stl
2 out of 24
adding --nozzle-diameter with value of 0.5 to cmd
adding --use-firmware-retraction with value of True to cmd
adding --use-volumetric-e with value of True to cmd
adding --vibration-limit with value of 5 to cmd
 slic3r --nozzle-diameter 0.5 --use-firmware-retraction --use-volumetric-e --vibration-limit 5 --output-filename-format [input_filename_base]_nozzle_diameter[nozzle_diameter]_use_firmware_retraction[use_firmware_retraction]_u

KeyboardInterrupt: 

In [45]:
%%sh
rm stl_files/*.gcode